In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import cv2

import torch.nn.functional as F
from skimage.transform import resize
import matplotlib.pyplot as plt
%matplotlib inline  

import random

from torchvision import datasets
from IPython.display import Image
from IPython.core.display import Image, display
from torchvision.utils import save_image

In [ ]:
batch_size = 256
learning_rate = 0.0001
num_epoch = 10

In [ ]:
# Load Data
dataset_train = datasets.ImageFolder(root='/content/drive/My Drive/AI/input/data/train', transform=transforms.Compose([
    transforms.Resize(64),
    transforms.ToTensor(), 
]))
dataset_test = datasets.ImageFolder(root='/content/drive/My Drive/AI/input/data/temp', transform=transforms.Compose([
    transforms.Resize(64),
    transforms.ToTensor(), 
]))
train_dataloader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size, shuffle=True)
len(dataset_train.imgs), len(train_dataloader)

In [ ]:
# Fixed input for debugging

fixed_x, _ = next(iter(test_dataloader))
save_image(fixed_x, 'real_image.png')

Image('real_image.png')

In [ ]:
class CNN(nn.Module):
    def __init__(self,num_feature=32):
        super(CNN,self).__init__()
        self.num_feature = num_feature
        
        self.layer = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=self.num_feature, kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(self.num_feature),
            nn.ReLU(),

            nn.Conv2d(self.num_feature,self.num_feature*2,3,1,1), #64
            nn.BatchNorm2d(self.num_feature*2),
            nn.ReLU(),
            nn.AvgPool2d(2,2),
            
            nn.Conv2d(self.num_feature*2,self.num_feature*4,3,1,1), #128
            nn.BatchNorm2d(self.num_feature*4),
            nn.ReLU(),

            nn.Conv2d(self.num_feature*4,self.num_feature*8,3,1,1), #256
            nn.BatchNorm2d(self.num_feature*8),
            nn.AvgPool2d(2,2),
            nn.ReLU(),
            
            nn.Conv2d(self.num_feature*8,self.num_feature*16,3,1,1), #512
            nn.BatchNorm2d(self.num_feature*16),
            nn.ReLU()
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(self.num_feature*4096,3000),
            nn.ReLU(),
            nn.Linear(3000,1000),
            nn.ReLU(),
            nn.Linear(1000,2)
        )       
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # Kaming Initialization
                init.kaiming_normal(m.weight.data)
                m.bias.data.fill_(0)
            elif isinstance(m, nn.Linear):
                # Kaming Initialization
                init.kaiming_normal(m.weight.data)
                m.bias.data.fill_(0)
        
        
    def forward(self,x):
        out = self.layer(x)
        out = out.view(x.size()[0],-1)
        out = self.fc_layer(out)
        return out

model = nn.DataParallel(CNN().cuda())

In [ ]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
check = []
for i in range(num_epoch):
    model.train()
    for j,[image,label] in enumerate(train_dataloader):
        img = Variable(image).cuda()
        img = img.float ()
        real_label= Variable(label).cuda()
        
        optimizer.zero_grad()
        output = model.forward(img)

        loss = loss_func(output,real_label)
        loss.backward()
        optimizer.step()
        
    succ_count = torch.FloatTensor([0])
    total = torch.FloatTensor([0])
    model.eval() 
    
    for image,label in test_dataloader:
        img = Variable(image,volatile=True).cuda()
        img = img.float ()
        real_label= Variable(label).cuda()

        output = model.forward(img)
        
        _, predicted = output.max(dim=1)
        # check data [Predicted_V, Original_V]
        check = ["Predicted_V : ",predicted, "Original_V : ", real_label]

        # 분자
        succ_count += torch.sum(predicted == real_label).float().cpu().data
        # 분모
        total += real_label.size(0)
    print("Test Data Accuracy: {}%".format(100*(succ_count/total).numpy()))
    if (succ_count/total).numpy() > 0.98:
        break

In [ ]:
print(check)

In [ ]:
class CAM():
    def __init__(self,model):
        self.gradient = []
        self.h = model.module.layer[-1].register_backward_hook(self.save_gradient)
        
    def save_gradient(self,*args):
        grad_input = args[1]
        grad_output= args[2]
        self.gradient.append(grad_output[0])
      
    def get_gradient(self,idx):
        return self.gradient[idx]
    
    def remove_hook(self):
        self.h.remove()
            
    def normalize_cam(self,x):
        x = 2*(x-torch.min(x))/(torch.max(x)-torch.min(x)+1e-8)-1
        x[x<torch.max(x)]=-1
        return x
    
    def visualize(self,cam_img,img_var):
        cam_img = resize(cam_img.cpu().data.numpy(),output_shape=(64,64))
        x = img_var[0,:,:].cpu().data.numpy()

        plt.subplot(1,3,1)
        plt.imshow(cam_img)

        plt.subplot(1,3,2)
        plt.imshow(x,cmap="gray")

        plt.subplot(1,3,3)
        plt.imshow(x+cam_img)
        plt.show()

    def deprocess_image(x):
        ## 평균을 0으로, 표준편차를 0.1로 하도록 normalize한다.
        x -= x.mean()
        x /= (x.std() + 1e-5)
        x *= 0.1

        ## [0, 1]사이로 클리핑한다.
        x += 0.5
        x = np.clip(x, 0, 1)

        ## 255를 곱해 RGB 값으로 바꾼다.
        x *= 255
        
        ## [0, 255]사이로 클리핑한 후 정수로 바꾼다. 
        x = np.clip(x, 0, 255).astype('uint8')
        return x

    
    def get_cam(self,idx):
        grad = self.get_gradient(idx)
        alpha = torch.sum(grad,dim=3,keepdim=True)
        alpha = torch.sum(alpha,dim=2,keepdim=True)
        
        cam = alpha[idx]*grad[idx]
        cam = torch.sum(cam,dim=0)
        cam = self.normalize_cam(cam)
        
        self.remove_hook()
        return cam

In [ ]:
cam = CAM(model)

for i,[image,label] in enumerate(test_dataloader):
    x = Variable(image).cuda()
    x = x.float()
    y_= Variable(label).cuda()
        
    output = model.forward(x)    
    
    for j in range(19):
        model.zero_grad()
        lab = y_[j].cpu().data
        output[j,lab].backward(retain_graph=True)

        out = cam.get_cam(j)
        cam.visualize(out,x[j])

    break